# 0. Imports and Setting up Anthropic API Client

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install python-dotenv

import os
import dotenv

dotenv.load_dotenv('/content/drive/MyDrive/.env')

True

In [3]:
# Load Prompts and Problem Description
prompt1_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt1_MathematicalModel.txt'
prompt2_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt2_PyomoCode.txt'
problem_desc_path = '/content/drive/MyDrive/Thesis/ProblemDescriptions/LP/LP3.txt'

prompt1_file = open(prompt1_path, "r")
prompt2_file = open(prompt2_path, "r")
problem_desc_file = open(problem_desc_path, "r")

prompt1 = prompt1_file.read()
print("Prompt 1:\n", prompt1)

prompt2 = prompt2_file.read()
print("Prompt 2:\n", prompt2)

problem_desc = problem_desc_file.read()
print("Problem Description:\n", problem_desc)

Prompt 1:
 Please write a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
Prompt 2:
 Please write a python pyomo code for this optimization problem.
Use sample data where needed.
Indicate where you use sample data.
Problem Description:
 The PRODA, S.A. industrial products firm has to face the problem of scheduling
the weekly production of its three products (P1, P2 and P3). These products are
sold to large industrial firms and PRODA, S.A. wishes to supply its products in
quantities that are more profitable for it.

Each product entails three operations contributing to the costs: smelting; mechanisation; assembly and
packaging. The smelting operations for products P1 and P2 could be subcontracted, but the smelting operation for product P3 requires special equipment, thus
preventing the use of subcontracts. PRODA also want to know, how much they should subcontract.

For product P1 the dir

In [4]:
!pip install anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00


In [5]:
# Importing Anthropic & Setting Headers
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

# 1. Prompt 1: Create Mathematical Model

In [6]:
message = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt1,
    max_tokens=4096,
    messages=[
        {"role": "user", "content": problem_desc}
    ]
)

response_p1 = message.content[0].text

In [7]:
# Print response
print(response_p1)

To solve the PRODA, S.A. production scheduling problem, we can formulate a linear programming model. Let's define the decision variables, parameters, objective function, and constraints.

Decision Variables:
- Let x1, x2, and x3 be the number of units produced for products P1, P2, and P3, respectively.
- Let s1 and s2 be the number of units subcontracted for smelting operation for products P1 and P2, respectively.

Parameters:
- Unit costs for smelting, mechanisation, and assembly and packaging for each product.
- Unit sales prices for each product.
- Time requirements for each operation per unit of product.
- Weekly capacities for each operation.

Objective Function:
Maximize weekly profit = (Revenue from sales) - (Production costs) - (Subcontracting costs)

Max Z = 1.50x1 + 1.80x2 + 1.97x3 - (0.30(x1-s1) + 0.20x1 + 0.30x1) - (0.50(x2-s2) + 0.10x2 + 0.20x2) - (0.40x3 + 0.27x3 + 0.20x3) - 0.50s1 - 0.60s2

Constraints:
1. Smelting capacity: 6(x1-s1) + 10(x2-s2) + 8x3 ≤ 8000
2. Mechanisa

# 2. Prompt 2: Write Code based on Mathematical Model

In [8]:
message2 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt2,
    max_tokens=4069,
    messages=[
        {"role": "user", "content": response_p1}
    ]
)

response_p2 = message2.content[0].text

In [9]:
# Print response
print(response_p2)

Here's the Python Pyomo code for the PRODA, S.A. production scheduling problem:

```python
from pyomo.environ import *

# Create the model
model = ConcreteModel()

# Define the decision variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)
model.x3 = Var(domain=NonNegativeReals)
model.s1 = Var(domain=NonNegativeReals)
model.s2 = Var(domain=NonNegativeReals)

# Define the objective function
model.profit = Objective(expr=1.50*model.x1 + 1.80*model.x2 + 1.97*model.x3 - (0.30*(model.x1-model.s1) + 0.20*model.x1 + 0.30*model.x1) - (0.50*(model.x2-model.s2) + 0.10*model.x2 + 0.20*model.x2) - (0.40*model.x3 + 0.27*model.x3 + 0.20*model.x3) - 0.50*model.s1 - 0.60*model.s2, sense=maximize)

# Define the constraints
model.smelting_capacity = Constraint(expr=6*(model.x1-model.s1) + 10*(model.x2-model.s2) + 8*model.x3 <= 8000)
model.mechanisation_capacity = Constraint(expr=6*model.x1 + 3*model.x2 + 8*model.x3 <= 12000)
model.assembly_capacity = Constraint(expr=3

# 4. Input Problem Data and Test Model Code

In [10]:
# Installing pyomo nd solver
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.6 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [11]:
from pyomo.environ import *

# Create the model
model = ConcreteModel()

# Define the decision variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)
model.x3 = Var(domain=NonNegativeReals)
model.s1 = Var(domain=NonNegativeReals)
model.s2 = Var(domain=NonNegativeReals)

# Define the objective function
model.profit = Objective(expr=1.50*model.x1 + 1.80*model.x2 + 1.97*model.x3 - (0.30*(model.x1-model.s1) + 0.20*model.x1 + 0.30*model.x1) - (0.50*(model.x2-model.s2) + 0.10*model.x2 + 0.20*model.x2) - (0.40*model.x3 + 0.27*model.x3 + 0.20*model.x3) - 0.50*model.s1 - 0.60*model.s2, sense=maximize)

# Define the constraints
model.smelting_capacity = Constraint(expr=6*(model.x1-model.s1) + 10*(model.x2-model.s2) + 8*model.x3 <= 8000)
model.mechanisation_capacity = Constraint(expr=6*model.x1 + 3*model.x2 + 8*model.x3 <= 12000)
model.assembly_capacity = Constraint(expr=3*model.x1 + 2*model.x2 + 2*model.x3 <= 10000)
model.subcontracting_constraint1 = Constraint(expr=model.s1 <= model.x1)
model.subcontracting_constraint2 = Constraint(expr=model.s2 <= model.x2)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Optimal solution:")
print("x1 =", model.x1.value)
print("x2 =", model.x2.value)
print("x3 =", model.x3.value)
print("s1 =", model.s1.value)
print("s2 =", model.s2.value)
print("Weekly profit =", model.profit.expr())

Optimal solution:
x1 = 0.0
x2 = 4000.0
x3 = 0.0
s1 = 0.0
s2 = 3200.0
Weekly profit = 3680.0


# 5. Correct The Model Code to Test Mathematical Model (if applicable)